In [1]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 755 kB/s eta 0:00:01


In [2]:
import gc
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
import numpy as np
import lightgbm as lgb
import time

训练集

In [3]:
user = pd.read_csv('data/train_preliminary/user.csv')

In [4]:
ad = pd.read_csv('output/features/train/ad_id_embedding_df.csv')
agg = pd.read_csv('output/features/train/aggregate_features_df.csv')
creative = pd.read_csv('output/features/train/creative_id_embedding_df.csv')
mode = pd.read_csv('output/features/train/mode_features_df.csv')

In [5]:
train = pd.merge(user, ad, how='outer')
del user, ad
gc.collect()
train = pd.merge(train, agg, how='outer')
del agg
gc.collect()
train = pd.merge(train, creative, how='outer')
del creative
gc.collect()
train = pd.merge(train, mode, how='outer')
del mode
gc.collect()
train.shape

(900000, 213)

测试集

In [6]:
ad = pd.read_csv('output/features/test/ad_id_embedding_df.csv')
agg = pd.read_csv('output/features/test/aggregate_features_df.csv')
creative = pd.read_csv('output/features/test/creative_id_embedding_df.csv')
mode = pd.read_csv('output/features/test/mode_features_df.csv')


test = pd.merge(ad, agg, how='outer')
del agg
gc.collect()
test = pd.merge(test, creative, how='outer')
del creative
gc.collect()
test = pd.merge(test, mode, how='outer')
del mode
gc.collect()
test.shape

(1000000, 211)

In [7]:
features = [ x for x in train.columns if x not in ['user_id', 'age', 'gender']]
len(features)

210

In [8]:
target1 = train['age'] - 1
target2 = train['gender'] - 1

In [9]:
params1 = {
#     'max_depth':14,
#     'num_leaves': 64,
#     'min_data_in_leaf': 20,
#     'min_child_samples':20,
    'objective': 'multiclassova',
    'num_class': 10,
    'learning_rate': 0.1,
    "boosting": "gbdt",
#         'max_bin': 255,
#     'bagging_fraction': 0.85,
#     "feature_fraction": 0.8,
#     "bagging_freq": 1,
#     "bagging_fraction": 0.85,
#     "bagging_seed": 23,
#     "metric": 'logloss',
#     "lambda_l1": 0.2,
#     "nthread": 4,
}

In [10]:
params2 = {
    'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.11
}

In [11]:
def lgb_model_2(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros(len(train))
    #测试集
    predictions_lgb = np.zeros(len(test))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 100,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration) # 每折 验证集的概率
        oof_lgb[valid_idx] = val_pred                    #整个  验证集的概率
        val_target = target2.iloc[valid_idx]             #验证集的真实标签
        val_pred =(val_pred>=0.499).astype('int')        #验证集的预测标签
        print("fold_{}：valid_acc：{}".format(fold_, accuracy_score(val_target, val_pred)))   #每折  验证集的ACC
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = (predictions_lgb>=0.499).astype('int')
    
    print('评分信息')
    print("ROC: {:<8.5f}".format(roc_auc_score(target2, oof_lgb)))
    valid_pre = (oof_lgb>=0.499).astype('int')
    gender_acc = accuracy_score(target2,valid_pre)
    print("accuracy: {:<8.5f}".format(gender_acc))
    
    print("run time：{}".format((time.clock() - start) / 120))
    return valid_pre, predictions_lgb

In [13]:
def lgb_model_10(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros((len(train), 10))
    #测试集
    predictions_lgb = np.zeros((len(test), 10))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 1000,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration)
        oof_lgb[valid_idx] = val_pred
        val_target = target2.iloc[valid_idx]
        val_pred = np.argmax(val_pred, axis=1)
        print("fold_{}：valid_F1：{}".format(fold_, f1_score(val_pred, val_target, average='macro')))
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = np.argmax(predictions_lgb, axis=1)
    
    print('评分信息')
    valid_pre = np.argmax(oof_lgb, axis=1)    
    print("F1:",f1_score(valid_pre,target,average='macro'))

    print("run time：{}".format(time))
    return valid_pre, predictions_lgb


gender

In [14]:
valid_pre_gender, pre_gender = lgb_model_2(5, 2020, train, test, features, target2, params2)

fold：0 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965422	valid_1's auc: 0.964594
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.965422	valid_1's auc: 0.964594
fold_0：valid_acc：0.9145777777777778
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965708	valid_1's auc: 0.963104
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.965708	valid_1's auc: 0.963104
fold_1：valid_acc：0.9137222222222222
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965536	valid_1's auc: 0.963845
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.965536	valid_1's auc: 0.963845
fold_2：valid_acc：0.9142555555555556
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965712	valid_1's auc: 0.96306
Did not

In [15]:
test['predicted_age'] = -1

In [16]:
test['predicted_gender'] = pre_gender + 1

In [17]:
test

,user_id,ad_id_embe_0_mean,ad_id_embe_0_min,ad_id_embe_0_max,ad_id_embe_1_mean,ad_id_embe_1_min,ad_id_embe_1_max,ad_id_embe_2_mean,ad_id_embe_2_min,ad_id_embe_2_max,...,creative_id_embe_31_min,creative_id_embe_31_max,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode,predicted_age,predicted_gender
0,3153317,0.137139,-1.878430,3.220181,1.279823,-0.064980,4.656413,-0.070853,-2.085783,3.817105,...,-5.615712,1.954202,142315,127841,0,7,1134,0,-1,1
1,3284714,0.818208,-3.812498,4.961826,2.206912,0.036472,8.533576,-1.001294,-4.515707,0.892850,...,-6.265074,2.443111,1,1,0,18,2623,6,-1,2
2,3491929,-0.681650,-4.806726,1.625325,0.832691,-3.986367,3.720694,-1.200181,-6.020298,0.415714,...,-4.806105,1.604750,1,1,0,5,4117,0,-1,1
3,3831238,0.193356,-5.168724,4.511687,0.732344,-1.038256,5.402612,-1.132856,-8.822957,1.229192,...,-5.751515,2.815079,1,1,0,3,25732,247,-1,1
4,3304452,-0.602799,-6.223625,4.005105,1.072692,-3.545419,5.287879,-0.110198,-7.142450,2.655369,...,-7.723289,2.583888,2639577,2271868,0,18,14681,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3274620,-2.106218,-5.886030,0.637769,-0.526008,-2.088032,1.057639,-1.574159,-3.882844,-0.436958,...,0.357159,5.542229,4145994,3556891,0,18,14682,88,-1,1
999996,3818829,0.385067,-0.067956,0.591475,-0.391493,-1.055110,-0.068664,-0.436491,-1.157278,-0.113634,...,0.203168,1.570777,3861244,3317841,0,18,14682,88,-1,1
999997,3380122,-0.753055,-3.139724,0.209324,-0.121105,-1.133710,1.247863,-1.247632,-3.281766,-0.122985,...,-0.025971,5.369463,4026829,3457126,0,18,57948,24,-1,2
999998,3996272,0.499637,0.179510,0.770862,0.441261,0.060094,1.112402,-0.341362,-1.112615,-0.032550,...,0.526207,2.531266,4050535,3476774,0,18,9058,6,-1,2


In [18]:
sub = test[['user_id', 'predicted_age', 'predicted_gender']]

In [19]:
sub

,user_id,predicted_age,predicted_gender
0,3153317,-1,1
1,3284714,-1,2
2,3491929,-1,1
3,3831238,-1,1
4,3304452,-1,1
...,...,...,...
999995,3274620,-1,1
999996,3818829,-1,1
999997,3380122,-1,2
999998,3996272,-1,2


In [20]:
sub.to_csv("sub/info+ad+creative_gender.csv",index=False,sep=',')

age

In [21]:
valid_pre_age, pre_age = lgb_model_10(5, 2020, train, test, features, target1, params1)

fold：0 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.5417	valid_1's multi_logloss: 1.59733
fold_0：valid_F1：0.023955296439846657
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.54104	valid_1's multi_logloss: 1.59922
fold_1：valid_F1：0.024035708870349358
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.54105	valid_1's multi_logloss: 1.59851
fold_2：valid_F1：0.02398465861528727
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.54179	valid_1's multi_logloss: 1.59826
fold_3：valid_F1：0.024300010733007558
fold：3 end...

In [22]:
test['predicted_gender'] = -1
test['predicted_age'] = pre_age + 1
sub = test[['user_id', 'predicted_age', 'predicted_gender']]
sub

,user_id,predicted_age,predicted_gender
0,3153317,3,-1
1,3284714,3,-1
2,3491929,4,-1
3,3831238,3,-1
4,3304452,3,-1
...,...,...,...
999995,3274620,6,-1
999996,3818829,3,-1
999997,3380122,6,-1
999998,3996272,3,-1


In [23]:
sub.to_csv("sub/5+info+ad+creative_age.csv",index=False,sep=',')

In [36]:
age=pd.read_csv("sub/5+info+ad+creative_age.csv")
gender=pd.read_csv("sub/info+ad+creative_gender.csv")

In [37]:
df=pd.merge(age,gender,on='user_id')

In [38]:
df.drop(['predicted_gender_x','predicted_age_y'],axis=1,inplace=True)

In [42]:
df.columns=['user_id','predicted_age','predicted_gender']

In [44]:
df.to_csv('sub/submission.csv',index=False)